In [2]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy import text
import os

In [10]:
# 업로드할 파일 경로 설정
file_path = os.path.abspath("../../datas/avatar_characters_20250421_112300.csv")

df = pd.read_csv(file_path, low_memory=False)
df.head()

,id,name,image,bio,personalInformation,politicalInformation,chronologicalInformation,gender,eyeColor,hairColor,skinColor
0,1,Aang,https://static.wikia.nocookie.net/avatar/image...,"{'alternativeNames': ['Aangy (by Koko)', 'Kuzo...","{'loveInterst': 'Katara (wife)', 'allies': ['A...","{'profession': ['Air Nomad culture teacher', '...","{'firstAppearance': '""The Boy in the Iceberg""'...",Male,Gray,Dark brown (typically shaved),Light
1,2,Appa,https://static.wikia.nocookie.net/avatar/image...,"{'alternativeNames': 'NA', 'nationality': 'Air...","{'loveInterest': 'His mother, Aang, Momo, Air ...","{'profession': '\n', 'position': 'NA', 'predec...","{'firstAppearance': '""The Boy in the Iceberg""'...",Male,Brown,White and brown,Fur
2,3,Momo,https://static.wikia.nocookie.net/avatar/image...,"{'alternativeNames': 'NA', 'nationality': 'Air...","{'loveInterest': 'Aang, Appa, Katara, Sokka, T...","{'profession': '\n', 'position': 'NA', 'predec...","{'firstAppearance': '""The Southern Air Temple""...",Male,Light green,White and brown,Fur
3,4,Katara,https://static.wikia.nocookie.net/avatar/image...,{'alternativeNames': ['Gran Gran (by her grand...,{'loveInterest': 'Aang (husband; widowed)\nJet...,"{'profession': '\n', 'position': ['Daughter of...","{'firstAppearance': '""The Boy in the Iceberg""'...",Female,Blue,Dark brown (white in old age),Brown
4,5,Sokka,https://static.wikia.nocookie.net/avatar/image...,{'alternativeNames': ['Wang Fire (while in the...,{'loveInterest': 'Suki (girlfriend)\nYue (girl...,"{'profession': '\n', 'position': ['Chieftain i...","{'firstAppearance': '""The Boy in the Iceberg""'...",Male,Blue,Dark brown (shaved on the sides),Brown


In [37]:
# DB 연결

# 1. 연결 정보 설정
pg_user = "airflow"
pg_pass = "airflow"
pg_host = "localhost"
pg_port = "5432"
pg_db   = "mlops"
db_url  = f"postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}"

# 2. SQLAlchemy 엔진 생성 
engine = create_engine(db_url)

# 3. DB 연결 테스트
try:
    with engine.connect() as conn:
        print("✅ DB 연결 성공")
except Exception as e:
    print(f"❌ DB 연결 실패: {e}")

# 4. 테이블 존재 확인
inspector = inspect(engine)
if "avatar_characters" in inspector.get_table_names():
    print("✅ 테이블 'avatar_characters' 존재함")
else:
    print("❌ 테이블 없음")


✅ DB 연결 성공
✅ 테이블 'avatar_characters' 존재함


In [90]:
# 5. 데이터 저장

# 넣을 컬럼만 추출
columns = ["name", "image", "gender", "eye_color", "hair_color", "skin_color"]

# INSERT 쿼리 문자열
insert_query = text(
    f"""
        INSERT INTO avatar_characters (name, image, gender, eye_color, hair_color, skin_color)
        VALUES (:name, :image, :gender, :eye_color, :hair_color, :skin_color)
    """
)

# DB 연결 후 데이터 삽입
with engine.begin() as conn:
    for _, row in df[columns].iterrows():
        conn.execute(insert_query, {
            "name": row["name"],
            "image": row["image"],
            "gender": row["gender"],
            "eye_color": row["eye_color"],
            "hair_color": row["hair_color"],
            "skin_color": row["skin_color"],
        })

print("✅ 데이터 삽입 완료")

✅ 데이터 삽입 완료


In [42]:
# 6. 데이터 확인

# DB 연결 후 데이터 조회
with engine.connect() as conn:
    query = "SELECT * FROM avatar_characters"
    result = conn.execute(text(query))
    df_result = pd.DataFrame(result.fetchall(), columns=result.keys())

print(len(df_result))
df_result.head()

39


,id,name,image,gender,eye_color,hair_color,skin_color
0,1,Aang,https://static.wikia.nocookie.net/avatar/image...,Male,Gray,Dark brown (typically shaved),Light
1,2,Appa,https://static.wikia.nocookie.net/avatar/image...,Male,Brown,White and brown,Fur
2,3,Momo,https://static.wikia.nocookie.net/avatar/image...,Male,Light green,White and brown,Fur
3,4,Katara,https://static.wikia.nocookie.net/avatar/image...,Female,Blue,Dark brown (white in old age),Brown
4,5,Sokka,https://static.wikia.nocookie.net/avatar/image...,Male,Blue,Dark brown (shaved on the sides),Brown
